#Test from Research Paper for The Jumping Spiders Team

Following the techniques for text-segmentation and parsing developed in [*Utilizing Artificial Intelligence for Text Segmentation from Images*](https://web.p.ebscohost.com/ehost/pdfviewer/pdfviewer?vid=0&sid=0531eae4-3f69-483c-b4a5-b8f728de4b78%40redis), I hope to find an effective way to preprocess, train, and test the model while yeilding high success rates.

> These tests were conducted by Alexia Rangel K.

---


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  9 22:07:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   40C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Data Collection and Input
Just used images that were collected from the Lepidorma library and had legible machine text



In [2]:
#Mounting the google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/data_folder/tjs-butterfly-imagetotext/
%ldir

/content/drive/MyDrive/Colab Notebooks/data_folder/tjs-butterfly-imagetotext
drwx------ 2 root      4096 Feb 10 21:32 Butterflies/
drwx------ 2 root      4096 Mar  1 13:57 preprocessed/
drwx------ 2 root      4096 Feb 11 01:13 train_images/


In [4]:
import pandas as pd #Dataframe stuff
import tensorflow as tf #Tensorflow is a machine learning dataset used for building Neural Nets
import os #Used to navigate through file structures
import cv2
from matplotlib import pyplot as plt
from sklearn import preprocessing
import numpy as np

There's first a few checks/reformatting we needed to make to the data to make sure it was ready for preprocessing/training. These only had to happen once.

In [5]:
#train_dir = 'train_images'

In [6]:
def checkExt (data_dir):
  #Quick Reviewing the Extensions
  count = 0;
  for image in os.listdir(data_dir):
    #For each image in the tjs directory
    image_path = os.path.join(data_dir, image)
    count = count + 1
    try:
      img = cv2.imread(image_path)
      ext = image_path.lower().endswith(('.jpg', '.jpeg'))    #ISSUE HERE FOR SOME REASON
      if not ext:
        print('Image not in ext list {}'.format(image_path))
    except Exception as e:
      print('Issue with image {}, {}'.format(image_path, e))
  print('Counted', count, 'images')

In [7]:
# Previous Operation: to check that all the files in data folder are a .jpg or .jpeg
# checkExt(butterfly_dir)
# checkExt(train_dir)

In [8]:
# Previous Operation: to add the path of the images to the end of each corresponding line
# image_files = sorted(os.listdir(train_dir))
# train_df_imagePath = train_df
# train_df_imagePath.reset_index(drop=True, inplace=True)
# train_df_imagePath['image_path'] = None

# for img in image_files:
#   img_path = os.path.join(train_dir, img)
#   img_name, ext= os.path.splitext(img)
#   train_df_imagePath.loc[train_df_imagePath['image_id'] == img_name, 'image_path'] = img_path

# train_df_imagePath.head(20)

In [9]:
# Previous Operation: saving the dataframe
# train_df_imagePath.to_csv('annot_path.csv')

Now onto our real stuff

In [10]:
# now we're loading this dataframe
text_df = pd.read_csv('annot_path.csv')
text_df.head(20)

,Unnamed: 0,id,image_id,bbox,utf8_string,image_path
0,0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,train_images/a4ea732cd3d5948a.jpg
1,1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,"[534.67, 64.68, 91.22, 38.19]",Sport,train_images/a4ea732cd3d5948a.jpg
2,2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,"[626.95, 63.62, 96.52, 31.82]",Watch,train_images/a4ea732cd3d5948a.jpg
3,3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,"[577.4, 141.87, 147.13, 43.1]",...period.,train_images/a4ea732cd3d5948a.jpg
4,4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,"[391.03, 163.9, 60.82, 38.65]",.,train_images/a4ea732cd3d5948a.jpg
5,5,a4ea732cd3d5948a_6,a4ea732cd3d5948a,"[455.64, 204.45, 21.97, 24.82]",400,train_images/a4ea732cd3d5948a.jpg
6,6,a4ea732cd3d5948a_7,a4ea732cd3d5948a,"[474.58, 234.95, 17.24, 24.25]",300,train_images/a4ea732cd3d5948a.jpg
7,7,a4ea732cd3d5948a_8,a4ea732cd3d5948a,"[506.48, 263.69, 25.42, 41.99]",15,train_images/a4ea732cd3d5948a.jpg
8,8,a4ea732cd3d5948a_9,a4ea732cd3d5948a,"[346.4, 177.05, 54.56, 38.48]",12,train_images/a4ea732cd3d5948a.jpg
9,9,a4ea732cd3d5948a_10,a4ea732cd3d5948a,"[364.49, 163.27, 19.82, 16.08]",60,train_images/a4ea732cd3d5948a.jpg


In [11]:
text_df['preprocessed_path'] = None;
for row in text_df.itertuples():
  name = row.image_id;
  text_df.loc[row.Index, 'preprocessed_path'] = f'preprocessed/{name}.jpg'

label encoding: https://www.geeksforgeeks.org/ml-label-encoding-of-datasets-in-python/

In [12]:
label_encoder = preprocessing.LabelEncoder()
text_df['numLabel']= label_encoder.fit_transform(text_df['utf8_string'])
max_label = np.max(text_df['numLabel'])

In [13]:
print(max_label)  #highest number label

179569


In [14]:
text_df.to_csv('annot_preprocessedpath.csv')
# text_df = pd.read_csv('annot_path.csv')

In [15]:
text_df.head(50)

,Unnamed: 0,id,image_id,bbox,utf8_string,image_path,preprocessed_path,numLabel
0,0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,93729
1,1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,"[534.67, 64.68, 91.22, 38.19]",Sport,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,109060
2,2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,"[626.95, 63.62, 96.52, 31.82]",Watch,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,121124
3,3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,"[577.4, 141.87, 147.13, 43.1]",...period.,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,7610
4,4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,"[391.03, 163.9, 60.82, 38.65]",.,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,7486
5,5,a4ea732cd3d5948a_6,a4ea732cd3d5948a,"[455.64, 204.45, 21.97, 24.82]",400,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,18656
6,6,a4ea732cd3d5948a_7,a4ea732cd3d5948a,"[474.58, 234.95, 17.24, 24.25]",300,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,17321
7,7,a4ea732cd3d5948a_8,a4ea732cd3d5948a,"[506.48, 263.69, 25.42, 41.99]",15,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,12860
8,8,a4ea732cd3d5948a_9,a4ea732cd3d5948a,"[346.4, 177.05, 54.56, 38.48]",12,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,11776
9,9,a4ea732cd3d5948a_10,a4ea732cd3d5948a,"[364.49, 163.27, 19.82, 16.08]",60,train_images/a4ea732cd3d5948a.jpg,preprocessed/a4ea732cd3d5948a.jpg,20673


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [17]:
#split the data into test and train
X = text_df.drop(['utf8_string', 'bbox', 'Unnamed: 0', 'id', 'image_id', 'preprocessed_path'], axis=1)
y = text_df[['preprocessed_path', 'utf8_string', 'bbox', 'numLabel']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [18]:
X_train.head(10)

,image_path,numLabel
19075,train_images/4f1eae3411d1602d.jpg,34386
490120,train_images/1c4ef149449f9f48.jpg,152182
418439,train_images/3a22833773160a8f.jpg,7486
762680,train_images/a6e8cf76da133544.jpg,117794
318905,train_images/b0ed5889747fcc34.jpg,7486
878846,train_images/8f912cd922b3e107.jpg,60721
530844,train_images/231edcd1d06a1c69.jpg,7486
1017609,train_images/fc0442b3a766bae3.jpg,14901
350407,train_images/3b279896dfe276a7.jpg,173769
658211,train_images/97312993bc725b46.jpg,88610


In [19]:
y_train.head(10)

,preprocessed_path,utf8_string,bbox,numLabel
19075,preprocessed/4f1eae3411d1602d.jpg,Baby,"[805.63, 452.79, 21.98, 39.71]",34386
490120,preprocessed/1c4ef149449f9f48.jpg,"men,","[418.29, 433.57, 43.6, 19.8]",152182
418439,preprocessed/3a22833773160a8f.jpg,.,"[246.56, 476.21, 32.27, 13.56]",7486
762680,preprocessed/a6e8cf76da133544.jpg,VIII,"[488.03, 405.22, 33.38, 35.58]",117794
318905,preprocessed/b0ed5889747fcc34.jpg,.,"[246.15, 180.43, 24.66, 5.6]",7486
878846,preprocessed/8f912cd922b3e107.jpg,GUEST,"[853.96, 286.09, 31.86, 10.79]",60721
530844,preprocessed/231edcd1d06a1c69.jpg,.,"[744.13, 228.75, 9.35, 13.2]",7486
1017609,preprocessed/fc0442b3a766bae3.jpg,2,"[36.15, 698.33, 9.8, 10.3]",14901
350407,preprocessed/3b279896dfe276a7.jpg,with,"[807.05, 389.57, 16.5, 8.19]",173769
658211,preprocessed/97312993bc725b46.jpg,Odieuvire,"[121.67, 974.76, 62.1, 12.5]",88610


#Pre-processing


*   image binarization using Otsu's method
*   image smoothing using Gaussian filter
*   image enhancement using Contrast Limited Adaptive Histogram Equalization (CLAHE) algorithm
*   crop Image

I'm watching [this video](https://www.youtube.com/watch?v=ADV-AjAXHdc) to do better ocr

### 2/21/25 Update:
Spent a lot of time reorganizing preprocessing, reading/watching resources on what is the best ways to preprocess for ocr, and trying out different methods to see what gives the best results. I found that any form of noise reduction, whether it be CLAHE, Dialating, Eroding, MorphologyEx, etc would make my images much more busy and hard to read.

I also read an article about [Adaptive Thresholding](https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html), which is much more useful for images of variable lighting (which is how my data is). Adaptive Thresholding determines the threshold of a pixel based on the region around it, which allows binarization to be more uniform. I used Adaptive Gaussian Thresholding. It worked a lot better than my previous attempts with Otsu's Binarization.

Other resources I want to look into:
- https://www.geeksforgeeks.org/bounding-box-prediction-using-pytorch/
- https://builtin.com/data-science/python-ocr
- https://www.youtube.com/watch?v=9FCw1xo_s0I
- https://github.com/wjbmattingly/ocr_python_textbook/blob/main/03_bounding_boxes_index-Copy1.ipynb
- https://www.reddit.com/r/deeplearning/comments/sxagv0/help_training_a_model_on_google_colab_and_my/

In [20]:
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from PIL import Image, ImageFilter
from google.colab.patches import cv2_imshow as show_img
from scipy.ndimage import gaussian_filter
from copy import deepcopy

In [21]:
def inverse(image):
  return cv2.bitwise_not(image)

In [22]:
def grayscale(image):
  return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [23]:
def binarization(image):
  # Apply Adaptive Thresholding: this is supposed to be better for images with variable lighting and conditions
  # https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html
  image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                              cv2.THRESH_BINARY,11,2)
  # _, image = cv2.threshold(image, 200, 230, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  return image

In [24]:
def noise_removal(image):
  kernal = np.ones((1, 1), np.uint8)
  image = cv2.dilate(image, kernal, iterations=1)
  kernal = np.ones((1, 1), np.uint8)
  image = cv2.erode(image, kernal, iterations=1)
  image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernal)
  image = cv2.medianBlur(image, 3)

  # Apply Gaussian filter (COMPARE TO MEDIANBLUR)
  # sigma = 2  # Standard deviation for Gaussian kernel (higher = more blur)
  # image = gaussian_filter(image, sigma=sigma)

  return image

In [25]:
def thin_font(image):
  image = cv2.bitwise_not(image)
  kernal = np.ones((2, 2), np.uint8)
  image = cv2.erode(image, kernal, iterations=1)
  image = cv2.bitwise_not(image)
  return image

In [26]:
def thick_font(image):
  image = cv2.bitwise_not(image)
  kernal = np.ones((2, 2), np.uint8)
  image = cv2.dilate(image, kernal, iterations=1)
  image = cv2.bitwise_not(image)
  return image

In [27]:
def blurring(image):
  # Apply Gaussian filter
  sigma = 0.5  # Standard deviation for Gaussian kernel (higher = more blur)
  return gaussian_filter(image, sigma=sigma)

In [28]:
def imgEnhancement(image):
  # Apply CLAHE Algorithm
  clahe = cv2.createCLAHE(clipLimit=5)
  return clahe.apply(image) + 10

In [29]:
def imgPreprocessing(img_path):
  image = cv2.imread(img_path)

  # image = inverse(image)
  # show_img(image)

  image = grayscale(image)
  # show_img(image)

  image = blurring(image)
  #show_img(image)

  # image = imgEnhancement(image)
  # show_img(image)

  image = binarization(image)
  #show_img(image)

  # image = noise_removal(image)
  # show_img(image)

  # image = thin_font(image)
  # show_img(image)

  # image = thick_font(image)
  # show_img(image)
  return (image)

In [30]:
# test = imgPreprocessing("train_images/4bf43a7b2a898044.jpg")
# test = imgPreprocessing("train_images/0a3f9715e6677a08.jpg")
test = imgPreprocessing("train_images/0b9e647fccd97986.jpg")
# test

In [31]:
def img_to_array(img_path):
  parts = img_path.split('/')
  img_name = parts[1]
  #Attempt to speed up stuff/do what wasn't done
  if not os.path.exists(f'preprocessed/{img_name}'):
    img_array = imgPreprocessing(img_path)
    cv2.imwrite(f'preprocessed/{img_name}', img_array)

In [32]:
# Test
# img_to_array("train_images/0b9e647fccd97986.jpg")

In [33]:
#preprocess training data
X_train['image_path'].apply(img_to_array)

,image_path
19075,None
490120,None
418439,None
762680,None
318905,None
...,...
131829,None
406610,None
3492,None
568149,None


In [34]:
#preprocess testing data
X_test['image_path'].apply(img_to_array)

,image_path
943560,None
651318,None
989191,None
954947,None
1045592,None
...,...
755263,None
158296,None
954952,None
984654,None


Checking that we preprocessed every image

In [35]:
#check how many images are in preprocessed
print(len([name for name in os.listdir('preprocessed/') if os.path.isfile(os.path.join('preprocessed/', name))]))

21778


In [36]:
#check compare to how many are in train_images (original)
print(len([name for name in os.listdir('train_images/') if os.path.isfile(os.path.join('train_images/', name))]))

25119


In [37]:
#to check that all the paths exist
print(X_train[X_train['image_path'] == None]);
print(X_test[X_test['image_path'] == None]);

Empty DataFrame
Columns: [image_path, numLabel]
Index: []
Empty DataFrame
Columns: [image_path, numLabel]
Index: []


In [38]:
# image = cv2.imread("preprocessed/8b44bfa1b5717879.jpg")
# image = image.astype(np.float32)
# image = tf.image.resize_with_pad(image, 256, 256) # resize image
# image = image.numpy()
# image = image.clip(0, 255).astype('uint8') #clip values to be between 0-255
# show_img(image)

# Training a model


In [47]:
def preprocess_image(img_path):
    img_path = img_path.decode('utf-8')
    img = cv2.imread(img_path)                  # Load image

     # Check if image loading was successful
    if img is None:
        print(f"Error loading image: {img_path}")  # Print error message
        return np.zeros((1024 ,768, 3), dtype=np.float32) # Return a dummy image to avoid stopping the training process

    img = img.astype(np.float32)
    img = cv2.resize(img, (768, 1024))
    return img

In [48]:
def create_dataset(df):
  image_path = df['preprocessed_path'].values
  labels = df['numLabel'].values

  # TensorFlow Dataset
  dataset = tf.data.Dataset.from_tensor_slices((image_path, labels))

  def load(path, label):
    img = tf.numpy_function(preprocess_image, [path], tf.float32)
    img.set_shape([1024 ,768, 3])
    return img, label

  # # Apply image loading and preprocessing
  dataset = dataset.map(load, num_parallel_calls=tf.data.AUTOTUNE)

  return dataset

In [41]:
tst = create_dataset(y_train)

In [42]:
X_train.head(5)

,image_path,numLabel
19075,train_images/4f1eae3411d1602d.jpg,34386
490120,train_images/1c4ef149449f9f48.jpg,152182
418439,train_images/3a22833773160a8f.jpg,7486
762680,train_images/a6e8cf76da133544.jpg,117794
318905,train_images/b0ed5889747fcc34.jpg,7486


In [43]:
y_train.head(5)

,preprocessed_path,utf8_string,bbox,numLabel
19075,preprocessed/4f1eae3411d1602d.jpg,Baby,"[805.63, 452.79, 21.98, 39.71]",34386
490120,preprocessed/1c4ef149449f9f48.jpg,"men,","[418.29, 433.57, 43.6, 19.8]",152182
418439,preprocessed/3a22833773160a8f.jpg,.,"[246.56, 476.21, 32.27, 13.56]",7486
762680,preprocessed/a6e8cf76da133544.jpg,VIII,"[488.03, 405.22, 33.38, 35.58]",117794
318905,preprocessed/b0ed5889747fcc34.jpg,.,"[246.15, 180.43, 24.66, 5.6]",7486


In [49]:
batch_size = 32

train_dataset = create_dataset(y_train).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = create_dataset(y_test).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = create_dataset(y_val).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [50]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1024, 768, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(max_label + 1, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
EPOCHS = 10
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

Epoch 1/10
19732/19732 ━━━━━━━━━━━━━━━━━━━━ 12317s 622ms/step - accuracy: 0.3196 - loss: 7.8527 - val_accuracy: 0.3204 - val_loss: 8.1874
Epoch 2/10
19732/19732 ━━━━━━━━━━━━━━━━━━━━ 11014s 558ms/step - accuracy: 0.3202 - loss: 7.5981 - val_accuracy: 0.3204 - val_loss: 8.1480
Epoch 3/10
19732/19732 ━━━━━━━━━━━━━━━━━━━━ 11007s 558ms/step - accuracy: 0.3202 - loss: 7.7248 - val_accuracy: 0.3204 - val_loss: 8.1991
Epoch 4/10
19732/19732 ━━━━━━━━━━━━━━━━━━━━ 11012s 558ms/step - accuracy: 0.3202 - loss: 7.7807 - val_accuracy: 0.3204 - val_loss: 8.2159
Epoch 5/10
 1814/19732 ━━━━━━━━━━━━━━━━━━━━ 2:30:41 505ms/step - accuracy: 0.3205 - loss: 7.7817

KeyboardInterrupt: 

In [ ]:
model.save('textmodel.keras')

In [91]:
model.evaluate(test_dataset)

6578/6578 ━━━━━━━━━━━━━━━━━━━━ 937s 142ms/step - accuracy: 0.3221 - loss: 8.1957


[8.2031831741333, 0.32184481620788574]



---



---



---


# After this is a lot of testing that isn't done, don't cross over!

Attempting to dynamically feed images into a model

In [ ]:
print(train_imageData.iloc[0:74])

In [ ]:
train_imageData.to_csv('annot_imageData.csv')

# Text Segmentation Algorithms


# Evaluations
